# Расчет проводимости трубопроводов с помощью МУК

In [1]:
import os
os.chdir("..")

from rectangular.rectangular import RectangleSeparator
from graph       import save_graph, customize_graph
from matplotlib  import pyplot as plt
from tube        import Tube
import numpy  as np
import pandas as pd

## Проверка УК

In [2]:
def print_tables(t, phi):
    #заполнение по столбцам
    #столбец - номер эмиттера
    #строчка - номер коллектор (внутри соответствующей грани)
    for side_1 in range(6):
        for side_2 in range(6):
            my_table = pd.DataFrame()
            for emitter_i in range(t.separator.find_num_cells(side_1)):
                my_table[str(emitter_i)] = phi[side_1][emitter_i][side_2]
            print(f"From {side_1}-th to {side_2}-th")
            display(my_table)
    return

def create_tube(a,b,L, cell):
    return Tube(RectangleSeparator(a,b,L,cell))

In [3]:
tube_1, tube_2 = create_tube(1,1,1,0.01), create_tube(1,5,1,0.01)

In [4]:
#phi_1 = tube_1.calc_angular_coeffs()

In [5]:
#phi_2 = tube_2.calc_angular_coeffs()

### Проверка замкнутости

In [6]:
def check_isolation(find_num, phi):
    sums = np.array([])
    for side in range(6):
        for emitter_i in range(find_num(side)):
            sums = np.append(sums, np.sum(phi[side][emitter_i]))
    return max(sums), np.mean(sums)

def create_isolation_graph(X : list):
    MAX_1, MAX_2 = [0]*len(X), [0]*len(X)
    MEAN_1,MEAN_2 = [0]*len(X), [0]*len(X)
    for i in range(len(X)):
        t_1, t_2 = create_tube(1,1,1,X[i]), create_tube(1,5,1,X[i])
        coeffs_1, coeffs_2 = t_1.calc_angular_coeffs(), t_2.calc_angular_coeffs()
        MAX_1[i], MEAN_1[i] = check_isolation(t_1.separator.find_num_cells, coeffs_1)
        MAX_2[i], MEAN_2[i] = check_isolation(t_2.separator.find_num_cells, coeffs_2)
    
    dirname = os.getcwd() #"MethodOfAngularCoefficients"
    folder_path = os.path.join(dirname, "tests", "Media")
        
    plt.plot(X,MAX_1)
    customize_graph("Зависимость свойства замкнутости", "Размер ячейки", "Макс значение")
    save_graph("max_isolation1.png", folder_path)
    plt.show()
    
    plt.plot(X,MAX_2)
    customize_graph("Зависимость свойства замкнутости", "Размер ячейки", "Макс значение")
    save_graph("max_isolation2.png", folder_path)
    plt.show()
    
    plt.plot(X,MEAN_1)
    customize_graph("Зависимость свойства замкнутости", "Размер ячейки", "Среднее значение")
    save_graph("mean_isolation1.png", folder_path)
    plt.show()
    
    plt.plot(X,MEAN_2)
    customize_graph("Зависимость свойства замкнутости", "Размер ячейки", "Среднее значение")
    save_graph("mean_isolation2.png", folder_path)
    plt.show()
    return

In [ ]:
create_isolation_graph([0.25, 0.04, 0.01, 0.00078125, 0.0044444444, 0.0025])

C:\Users\ASUS\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


### Проверка взаимности

In [ ]:
def check_mutuality(t, phi):
    for i in range(1, len(phi)):
        for j in range(1, len(phi[i])):
            if round(phi[i][j],5) != round(phi[j][i],5):
                print("Свойство взаимности не выполнено для i =", i, "и j =", j)
                break 
    print("Проверка взаимности a =", t.a, "b =", t.b, "площади ячейки =", t.cell, "закончена")
    print("\n")   
    return

In [ ]:
check_mutuality(tube_1, phi_1)
check_mutuality(tube_2, phi_2)

### Проверка аддитивности

In [ ]:
def check_additivity(t, phi):
    for i in range(1,len(phi)):
        for j in range(1,len(phi[i])):
            if i == j:
                break
            new_phi = t.check_add(i,j)
            if round(new_phi,5) != round(phi[i][j],5):
                print("Свойство аддитивности не выполнено для i=", i, "и для j=", j)
    print("Проверка аддитивности для a =", t.a, "b =", t.b, "площади ячейки =", t.cell, "закончена")
    print("\n")
    return 

In [ ]:
check_additivity(tube_1, phi_1)
check_additivity(tube_2, phi_2)